<a href="https://colab.research.google.com/github/singhraj00/langchain-tutorial/blob/main/Langchain_Runnables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Langchain Aam-Jindagi

### we have create an fake llm, prompt class and after connect both them.

In [ ]:
import random

class FakeLLM:

  def __init__(self):
    print('LLM Created')

  def predict(self,prompt):
    response_list=[
        "delhi is the capital of india.",
        "IPL is a t20 league.",
        "AI stands for artificial intelligence"
    ]
    return {'response':random.choice(response_list)}

## test fake llm

In [ ]:
llm = FakeLLM()
llm.predict('what is the capital of india?')

LLM Created


{'response': 'delhi is the capital of india.'}

## craete an fake prompt

In [ ]:
class FakePrompt:
  def __init__(self,template,input_variables):
    self.template=template
    self.input_variables=input_variables

  def format(self,input_dict):
    return self.template.format(**input_dict)

## test fake prompt

In [ ]:
template=FakePrompt(
    template="write an {length} poem about {topic}",
    input_variables=['length','topic']
)

prompt = template.format({
   'length':'short',
    'topic':'AI'
})
prompt

'write an short poem about AI'

In [ ]:
llm = FakeLLM()
llm.predict(prompt)

LLM Created


{'response': 'IPL is a t20 league.'}

## Combined Them For Create an FAKELLMCHAIN

In [ ]:
class FakeLLMChain:
  def __init__(self,llm,prompt):
    self.llm=llm
    self.prompt=prompt

  def run(self,input_dict):
      final_prompt = self.prompt.format(input_dict)
      result = self.llm.predict(final_prompt)
      return result['response']

## Test Chain

In [ ]:
template=FakePrompt(
    template="write an {length} poem about {topic}",
    input_variables=['length','topic']
)

In [ ]:
llm = FakeLLM()

LLM Created


In [ ]:
chain = FakeLLMChain(llm,template)
chain.run({
    'length':'short',
    'topic':'AI'
})

'IPL is a t20 league.'

## Langchain Mentos--Jindagi

### concepts of Runnable Class

### Runnable is nothing , it's just an abstract class, whivh used to builds every componts in lagchain.

In [ ]:
from abc import ABC, abstractmethod

class Runnable(ABC):
    @abstractmethod
    def invoke(self,input):
        pass

## Re-Create FakeLLM Using Runnable

In [ ]:
class FakeLLM(Runnable):
  def __init__(self):
    print('llm created')

  def invoke(self,prompt):
    response_list=[
        "delhi is the capital of india.",
        "IPL is a t20 league.",
        "AI stands for artificial intelligence"
    ]
    return {'response':random.choice(response_list)}

    ## previous - in aam-jindagi
    def predict(self,prompt):
      response_list=[
        "delhi is the capital of india.",
        "IPL is a t20 league.",
        "AI stands for artificial intelligence"
     ]
      return {'response':random.choice(response_list)}


## Re-Create FakePrompt Class

In [ ]:

class FakePrompt(Runnable):
  def __init__(self,template,input_variables):
    self.template=template
    self.input_variables=input_variables

  ## mentos - jindagi

  def invoke(self,input_dict):
    return self.template.format(**input_dict)

  ## aam - jindagi
  def format(self,input_dict):
    return self.template.format(**input_dict)



## Create FakeStringOutputParser Class

In [ ]:
class FakeStrOutputParser(Runnable):
  def __init__(self):
    pass

  def invoke(self,input_data):
    return input_data['response']

## Create an class RunnableConnector (LLMChain)

In [ ]:
class FakeRunnableConnector(Runnable):
  def __init__(self,runnable_list):
    self.runnable_list = runnable_list

  def invoke(self,input_data):
    ## sequential chain
    for runnable in self.runnable_list:
      input_data = runnable.invoke(input_data)
    return input_data


In [ ]:
template = FakePrompt(
    template="write an {length} poem about {topic}",
    input_variables=['length','topic']
)

In [ ]:
llm = FakeLLM()

llm created


In [ ]:
chain = FakeRunnableConnector([template,llm,FakeStrOutputParser()])
chain.invoke({
    'length':'short',
    'topic':'AI'
})

'delhi is the capital of india.'

## A Chain is also Runnable , if you want to also combined then two chains between them.

In [ ]:
template1= FakePrompt(
    template='write me a joke about {topic}',
    input_variables=['topic']
)
template2= FakePrompt(
    template='Explain the following joke {response}',
    input_variables=['response']
)

In [ ]:
llm = FakeLLM()

llm created


In [ ]:
parser = FakeStrOutputParser()

In [ ]:
chain1 = FakeRunnableConnector([template1,llm])
chain2 = FakeRunnableConnector([template2,llm,parser])

In [ ]:
final_chain = FakeRunnableConnector([chain1,chain2])

In [ ]:
final_chain.invoke({'topic':'ipl'})

'delhi is the capital of india.'

## Runnables In LangChain

## Types Of Runnables
- Task-Specific Runnables
- Runnable-Primitives

## Task-Specific Runnables

### Definitions : These are core LangChain components that have been converted into runnables so they can be used in pipelines.

### Purpose: Perform task-specific operation like LLM calls, prompting, retrieval etc.

### Examples:
- ChatOpenAI - Runs as LLM Model
- PromptTemplate - Formats prompts dynamically
- Retrievar - Retrieves relevant documents

## Runnable Primitives

### Definitios: These are fundamentals building blocks for structuring execution logic in AI workflows.

### Purpose: They help orchestrate execution by defining how different runnables interact (sequentially, in parallel, conditionally etc.)

### Examples:
- RunnableSequence
- RunnableParallel
- RunnableBranch
- RunnablePassthroug
- Ru8nnableLambda

## RunnableSequence

### RunnableSequence is a sequential chain of runnables in LangChain that executes each step one after another, passing the output of one step the input as next. it is useful whenyou need to compose multiple runnables together in a structuring workflow.

## Code Example

In [114]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template = PromptTemplate(
    template='write me a joke about {topic}',
    input_variables = ['topic']
)

parser = StrOutputParser()

chain = RunnableSequence(template,model,parser)

print(chain.invoke({'topic':'cricket'}))


Why did the cricket go to the doctor?

Because it had a bug in its system, and it wasn't just a googly.


## Code Example 2

In [115]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template1 = PromptTemplate(
    template='write me a joke about {topic}',
    input_variables = ['topic']
)

template2 = PromptTemplate(
    template='Explain the following joke in {response}',
    input_variables = ['response']
)



parser = StrOutputParser()

chain1 = RunnableSequence(template1,model,parser)
chain2 = RunnableSequence(template2,model,parser)

final_chain = RunnableSequence(chain1,chain2)

print(final_chain.invoke({'topic':'cricket'}))


A clever joke. Let's break it down:

The joke starts by setting up a familiar scenario: "Why did the cricket go to the doctor?" This implies that the cricket is ill or has a problem.

The punchline is: "Because it had a bug in its system, and it wasn't just a googly."

Here, "bug in its system" is a play on words. In computing, a "bug" refers to an error or a glitch in a system. However, in this joke, "bug" also refers to an insect, which is a clever connection to the fact that the subject is a cricket (an insect).

The second part of the punchline, "and it wasn't just a googly," is a reference to the sport of cricket. In cricket, a "googly" is a type of delivery (a ball thrown by the bowler) that is designed to deceive the batsman. It's a clever and tricky delivery.

So, the joke is saying that the cricket went to the doctor because it had a bug (insect) in its system, but it's not just a simple "googly" (a tricky delivery in cricket). The joke relies on the double meaning of "bug" an

## RunnableParallel

### The RunnableParallel object allows us to define multiple values and operations, and run them all in parallel.

## Code Example

### We want to generate an twitter or linkedin post on specific topic in parallel.

In [128]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel,RunnableSequence

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template1 = PromptTemplate(
    template='generate an tweet about {topic}',
    input_variables = ['topic']
)

template2 = PromptTemplate(
    template='generate an linkedin post about {topic}',
    input_variables = ['topic']
)

parser = StrOutputParser()

parallel_chain= RunnableParallel({
    "tweet":RunnableSequence(template1,model,parser),
    "linkedin":RunnableSequence(template2,model,parser)
})

result = parallel_chain.invoke({'topic':'AI'})

print('Tweet: ',result['tweet'])
print('Linkedin: ',result['linkedin'])


final_chain.get_graph().print_ascii()

Tweet:  "AI is revolutionizing the world, from healthcare to finance, and beyond. What are your thoughts on the future of artificial intelligence? Will it enhance or disrupt our daily lives? #AI #ArtificialIntelligence #FutureTech"
Linkedin:  **The Future of Work: How AI is Revolutionizing Industries**

As we continue to navigate the ever-changing landscape of technology, one thing is clear: Artificial Intelligence (AI) is no longer just a buzzword, but a reality that's transforming the way we work and live.

From automating mundane tasks to analyzing complex data sets, AI is unlocking new levels of efficiency, productivity, and innovation across industries. Whether it's in healthcare, finance, or marketing, AI is helping professionals make better decisions, faster and with greater accuracy.

But what does this mean for the future of work? As AI takes on more routine and repetitive tasks, it's freeing up human talent to focus on high-value tasks that require creativity, empathy, and pr

## RunnablePassthrough

### Runnable to passthrough inputs unchanged or with additional keys.

In [123]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableSequence,RunnableParallel,RunnablePassthrough

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template1 = PromptTemplate(
    template='write me a joke about {topic}',
    input_variables = ['topic']
)

template2 = PromptTemplate(
    template='Explain the following joke in {response}',
    input_variables = ['response']
)



parser = StrOutputParser()


chain1 = RunnableSequence(template1,model,parser)
chain2 = RunnableParallel({
    "joke_gen":RunnablePassthrough(),
    "joke_exp":RunnableSequence(template2,model,parser)
})


final_chain = RunnableSequence(chain1,chain2)

result = final_chain.invoke({'topic':'cricket'})

print('Joke: ',result['joke_gen'])
print('Explanation: ',result['joke_exp'])


final_chain.get_graph().print_ascii()

Joke:  Why did the cricket go to the doctor?

Because it had a bug in its system, and it wasn't just a googly.
Explanation:  A clever joke. Let's break it down:

The joke starts by setting up a familiar scenario: "Why did the cricket go to the doctor?" This implies that the cricket is ill or has a problem.

The punchline is: "Because it had a bug in its system, and it wasn't just a googly."

Here, "bug in its system" is a play on words. In computing, a "bug" refers to an error or a glitch in a system. However, in this joke, "bug" also refers to an insect, which is a clever connection to the fact that the subject is a cricket (an insect).

The second part of the punchline, "and it wasn't just a googly," is a reference to cricket (the sport). In cricket, a "googly" is a type of delivery bowled by a spin bowler, which turns in the opposite direction to what the batsman expects. The joke is saying that the cricket's problem (the "bug") is not just a tricky or unexpected situation (like a g

## RunnableLambda

### RunnableLambda converts a python callable into a Runnable.

### CodeExample

In [137]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel,RunnableLambda,RunnableSequence,RunnablePassthrough

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template = PromptTemplate(
    template='write me a joke about {topic}',
    input_variables = ['topic']
)


def count_word(text):
  return len(text.split())

parser = StrOutputParser()

joke_gen_chain = RunnableSequence(template,model,parser)

joke_exp_chain = RunnableParallel({
    "joke":RunnablePassthrough(),
    ## runnable lambda used to convert python function to runnable lambda
    "word_count":RunnableLambda(count_word)
    ## second way
    # "word_count":(lambda x:len(x.split()))
})


final_chain = RunnableSequence(joke_gen_chain,joke_exp_chain)

result = final_chain.invoke({'topic':'cricket'})

print('joke: ',result['joke'])
print('word count: ',result['word_count'])


joke:  Why did the cricket go to the doctor?

Because it had a bug in its system, and it wasn't just a googly.
word count:  22


## RunnableBranch

### Runnable that selects which branch to run based on a condition.

## Code Example

In [144]:
from langchain_groq import ChatGroq
from google.colab import userdata
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableLambda,RunnableSequence,RunnablePassthrough,RunnableBranch

API_KEY = userdata.get('GROQ_API_KEY')

model = ChatGroq(model_name='llama-3.3-70b-versatile',api_key=API_KEY,temperature=0)

template1 = PromptTemplate(
    template='generate an detail report about {topic}',
    input_variables = ['topic']
)

template2 = PromptTemplate(
    template='Summarize the following report in {response}',
    input_variables = ['response']
)



parser = StrOutputParser()

report_gen_chain = RunnableSequence(template1,model,parser)

branch_chain= RunnableBranch(
    (lambda x: len(x.split())>200, RunnableSequence(template2,model,parser)),
    RunnablePassthrough()
)

final_chain = RunnableSequence(report_gen_chain,branch_chain)

result = final_chain.invoke({'topic':'Russia Vs Ukraine'})

print('Report: ',result)

final_chain.get_graph().print_ascii()


Report:  The conflict between Russia and Ukraine is a complex and ongoing issue that has been escalating since 2014. The crisis has its roots in a mix of historical, cultural, and economic factors, and has had significant implications for regional and global security. This report aims to provide an in-depth analysis of the conflict, including its causes, key events, and current situation, in order to understand the complexities of the issue and identify potential solutions. The report will examine the historical background of the conflict, the causes of the conflict, key events, and the current situation, as well as the humanitarian impact and international response, in order to provide a comprehensive understanding of the conflict and its implications.
  +-------------+    
  | PromptInput |    
  +-------------+    
          *          
          *          
          *          
+----------------+   
| PromptTemplate |   
+----------------+   
          *          
          *     